In [1]:
import pandas as pd
from collections import OrderedDict
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


Bad key "text.kerning_factor" on line 4 in
C:\Users\MARCELO\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [10]:
'''
EXTERNAL KERNEL
'''
google_colab = False
kaggle = False

'''
CUDA
'''
cuda = False

'''
DATA REPRESENTATION

1 => SINGLE READ | 2 => ADD FEATURES | 3 => WINDOW TO FEATURES
'''
DATA_REPRESENTATION = 2

'''
DOWNSAMPLE FACTOR

1 => 10hz *original rate* | 2 => 5Hz | 5 => 2Hz | 10 => 1hz
'''
DOWNSAMPLE_FACTOR = 5

'''
WINDOWS LENGHT

* needs divisor by datapoints target
* considering downsample factor = 5

1 => WINDOW DISABLED | 2 => 1 second | 4 => 2 seconds | 10 => 5 seconds | 20 => 10 seconds | 200 => 100 seconds *full flight*
'''
WINDOW_LENGHT =  1

'''
LIMITADOR

Quantity of samples in the execution of the tests.
'''
LIMITADOR = 20000

'''
LOSS FACTOR [0,1]

Ignores outliers in calculating the stats of losses in regenerated data.
'''
LOSS_FACTOR = 0.96

'''
TRAIN_SIZE [0,1]

Percentage of samples to be trained
'''
TRAIN_SIZE = 0.8

'''
OUTPUT_FILE_NAME

File with output results
'''
OUTPUT_FILE_NAME = 'output-dr_' + str(DATA_REPRESENTATION) + '-ts_' + str(TRAIN_SIZE) + '-lf_' + str.replace(str(LOSS_FACTOR), '.', '') + '-limit_' + str(LIMITADOR) + '-wl_' + str(WINDOW_LENGHT) + '.txt'

'''
PATH_OUTPUTS

'''
PATH_OUTPUTS = './outputs/'

'''
PATH_DATASET

'''
PATH_DATASET = '../../dataset/original/'

'''
FLUSH FILE

If output results file is ON
'''
FLUSH_FILE = True

In [3]:
if google_colab:
    from google.colab import drive

    drive.mount('/content/drive')
    path = '/content/drive/My Drive/ACADÊMICO/MESTRADO/DISSERTAÇÃO/CHAPTERS/5 EXPERIMENTO/dataset/data_representation_1'
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(path+'/F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(path+'/F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(path+'/F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(path+'/F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(path+'/F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(path+'/F16_DS3_fault4_fixedposition.csv', header=None),
    }
elif kaggle:
    !conda install -y gdown
    !gdown --id 1G88okIVmdcgLFlmd7rDRhHvHv98yK3UB
    !gdown --id 1fX3utfHMjwKTt7IW4D01bnm-hv88yzrJ
    !gdown --id 1yUG3R5zK2AIxtS9Q4Fk-udkKBZeYShgb
    !gdown --id 1OBRDtuqNEZ-3Z-q0helWh2xGiAxeLACH
    !gdown --id 17oDi60sWYsWHHxzj2aA9m6ARm8zQ81m_
    !gdown --id 1jKEK4s5sYJh8PHtpHeV8ABOsHjuB26RA
else:
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault4_fixedposition.csv', header=None),
    }

In [4]:
dict_ds = dict_ds_original.copy()

if dict_ds['data_ds3_normal_t1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0 or dict_ds['data_ds3_fault1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0:
    raise Exception('Needs to be ?shape? divisor')

for n, dataset_name in enumerate(dict_ds):
    dataset = dict_ds[dataset_name].to_numpy()

    downsampled = dataset[::DOWNSAMPLE_FACTOR]

    x, y = downsampled.shape

    # resample
    dict_ds[dataset_name] = pd.DataFrame(downsampled.reshape((int(x/WINDOW_LENGHT),y*WINDOW_LENGHT)))


In [5]:
# ADD COLUMNS WITH DIFF PREVIOUS VALUES

if (DATA_REPRESENTATION == 2):
    frame_size = int(1000/DOWNSAMPLE_FACTOR)

    for n, dataset_name in enumerate(dict_ds):
        dataset = dict_ds[dataset_name].to_numpy()

        dimension = dataset.shape[1]
        samples = dataset.shape[0]

        # GENERATE NEW DIMENSIONS
        dataset = np.concatenate((dataset, np.zeros((samples,dimension))), axis=1)

        for f in np.arange(0,int(samples/frame_size)):
            # OBTAIN THE FRAME FLIGHT
            frame = dataset[f*frame_size:(f+1)*frame_size, 0:dimension]

            # CALCULATE DIFFERENCE
            chunk = np.diff(frame, axis=0)

            # DONT CALCULATE THE DIFFERENCE FOR EACH FIRST TIMESTEP
            chunk = np.insert(chunk, 0, frame[0, 0:dimension], axis=0)

            # UPDATE DATASET WITH NEW FRAME INTO NEW DIMENSIONS
            dataset[f*frame_size:(f+1)*frame_size,dimension:dimension*2] = chunk

        dict_ds[dataset_name] = pd.DataFrame(dataset)



In [6]:
ss = StandardScaler()

data_ds3_t1_normal = dict_ds['data_ds3_normal_t1_original']
data_ds3_t2_normal = dict_ds['data_ds3_normal_t2_original']
data_ds3_fault1 = dict_ds['data_ds3_fault1_original']
data_ds3_fault2 = dict_ds['data_ds3_fault2_original']
data_ds3_fault3 = dict_ds['data_ds3_fault3_original']
data_ds3_fault4 = dict_ds['data_ds3_fault4_original']

# fit values
ss.partial_fit(data_ds3_t1_normal)
ss.partial_fit(data_ds3_t2_normal)
ss.partial_fit(data_ds3_fault1)
ss.partial_fit(data_ds3_fault2)
ss.partial_fit(data_ds3_fault3)
ss.partial_fit(data_ds3_fault4)

# transform values
data_ds3_t1_normal = ss.transform(data_ds3_t1_normal)
data_ds3_t2_normal = ss.transform(data_ds3_t2_normal)
data_ds3_fault1 = ss.transform(data_ds3_fault1)
data_ds3_fault2 = ss.transform(data_ds3_fault2)
data_ds3_fault3 = ss.transform(data_ds3_fault3)
data_ds3_fault4 = ss.transform(data_ds3_fault4)

# append normal labels
data_ds3_t1_normal = np.append(data_ds3_t1_normal, np.zeros((data_ds3_t1_normal.shape[0],1)), axis = 1)
data_ds3_t2_normal = np.append(data_ds3_t2_normal, np.zeros((data_ds3_t2_normal.shape[0],1)), axis = 1)

# append fault labels
def generate_fault_label(dataset, fault_label):
    labels = np.array([[fault_label]]*dataset.shape[0])

    return labels

data_ds3_fault1 = np.append(data_ds3_fault1, generate_fault_label(data_ds3_fault1, 1), axis = 1)
data_ds3_fault2 = np.append(data_ds3_fault2, generate_fault_label(data_ds3_fault2, 2), axis = 1)
data_ds3_fault3 = np.append(data_ds3_fault3, generate_fault_label(data_ds3_fault3, 3), axis = 1)
data_ds3_fault4 = np.append(data_ds3_fault4, generate_fault_label(data_ds3_fault4, 4), axis = 1)

In [7]:
dimension = data_ds3_t2_normal.shape[1]-1

# FUNCTIONS AND CLASSES
class Autoencoder(nn.Module):
    def __init__(self, encode_l, decode_l):
        super().__init__()
        self.encoder = nn.Sequential(encode_l)
        self.decoder = nn.Sequential(decode_l)

    def forward(self, x):
        return self.decoder(self.encoder(x))

def run_train(net, train_loader, num_epochs, optimizer, loss_func):
    train_loss = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        losses = []
        for n, (real_samples, _) in enumerate(train_loader):

            net.zero_grad()

            ### forward ###
            if cuda:
                output = net(real_samples.type(torch.FloatTensor).cuda())
                loss = loss_func(output, real_samples.type(torch.FloatTensor).cuda())
            else:
                output = net(real_samples.type(torch.FloatTensor))
                loss = loss_func(output, real_samples.type(torch.FloatTensor))

            running_loss += loss.item()
            losses.append(loss.item)

            ### backward ###
            loss.backward()
            optimizer.step()

        step_loss = running_loss / len(train_loader)
        train_loss.append(step_loss)

        ### log ###
        #print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss))

    return net, output, train_loss, losses

def generate_ground_truth(data_test):
    ground_truth = []

    for j, x in enumerate(data_test):

        if x.shape[0] == dimension: # X_test
            ground_truth.append(0)
        else: # % others: get column label
            if x[-1] != 0:
                ground_truth.append(1)
            else:
                ground_truth.append(0)

    return ground_truth

def generate_losses(data_test, net, loss_function):
    losses = []
    #regenerate_data = np.zeros((1, dimension))
    for j, x in enumerate(data_test):

        if x.shape[0] > dimension:
            x = x[:-1]

        real = x.reshape(1,-1).astype(np.float32)

        #regenerate_data[j] = regenerate.cpu().detach().numpy()

        if cuda:
            regenerate = net(torch.from_numpy(real).cuda())
            loss_ae = loss_function(regenerate, torch.from_numpy(real).cuda()).item()
        else:
            regenerate = net(torch.from_numpy(real))
            loss_ae = loss_function(regenerate, torch.from_numpy(real)).item()

        losses.append(loss_ae)

    return losses

def generate_y_hat(losses, loss_threshold):
    y_hat = []

    for l in losses:
        if l < loss_threshold:
            y_hat.append(0)
        else:
            y_hat.append(1)

    return y_hat

def tester(data_test, net, loss_function, loss_threshold = 1):

    ground_truth = []
    losses = []

    for n, dataset_name in enumerate(data_test):
        dataset = data_test[dataset_name]

        ground_truth = ground_truth + generate_ground_truth(dataset)

        losses = losses + generate_losses(dataset, net, loss_function)

    y_hat = generate_y_hat(losses, loss_threshold)

    return confusion_matrix(ground_truth, y_hat, normalize='true'), losses, ground_truth, y_hat

def generate_encode_decode_layers(layers, output_layer):
    od_encode = []
    od_decode = []
    # encode
    for _, layer in enumerate(layers):
        n = _ + 1

        if (len(layers) == n):
            break

        od_encode.append(('l'+str((len(od_encode)+1)), nn.Linear(layers[_],layers[n])))

        if (len(layers) != n+1):
            od_encode.append(('l'+str((len(od_encode)+1)), nn.ReLU()))

    # decode
    layers.reverse()
    for _, layer in enumerate(layers):
        n = _ + 1

        if (len(layers) == n):
            break

        od_decode.append(('l'+str((len(od_decode)+1)), nn.Linear(layers[_],layers[n])))

        if (len(layers) != n+1):
            od_decode.append(('l'+str((len(od_decode)+1)), nn.ReLU()))
        else:
            od_decode.append(('l'+str((len(od_decode)+1)), output_layer))

    return OrderedDict(od_encode), OrderedDict(od_decode)

def train(layers, last_layer, lr, epochs, batch_size, X_train, optim, loss_fnc, net = []):
    encode_l, decode_l = generate_encode_decode_layers(layers, last_layer)

    if (net == []):
        net = Autoencoder(encode_l, decode_l)

    if cuda:
        net.cuda()

    if (optim == 'ADAM'):
        optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=1e-5)
    elif(optim == 'SGD'):
        optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    elif(optim == 'RMSprop'):
        optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)

    torch.manual_seed(111)

    # sets
    train_set = [
        (X_train, X_train) for i in range(len(X_train))
    ]

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, shuffle=False
    )

    # train
    net, output, loss, losses = run_train(net, train_loader, epochs, optimizer, loss_fnc)

    return net, output, loss, losses, loss_fnc

def test(X_train, X_test, net, loss_function, phi = 0):

    faults = {'F1': data_ds3_fault1[:LIMITADOR],
              'F2': data_ds3_fault2[:LIMITADOR],
              'F3': data_ds3_fault3[:LIMITADOR],
              'F4': data_ds3_fault4[:LIMITADOR]}

    datas = {'X_TRAIN':X_train,
             'X_TEST':X_test[:LIMITADOR],
             'NORMAL 2':data_ds3_t2_normal[:LIMITADOR]}

    datas.update(faults)

    for n, dataset_name in enumerate(datas):
        print('Losses ' + dataset_name , file=log)

        if (dataset_name == 'X_TRAIN'):
            losses = generate_losses(datas[dataset_name], net, loss_function)
            np_losses = np.sort(np.array(losses))
            losses = np_losses[:int(len(np_losses)*LOSS_FACTOR)]
        else:
            losses = [1]

        print("mu: ", np.mean(losses), file=log)
        print("std: ", np.std(losses, ddof=1, dtype=np.float64), file=log)

        phi_test = np.mean(losses) + np.std(losses, ddof=1, dtype=np.float64)

        print("phi: ", phi_test, file=log)

        if (dataset_name == 'X_TRAIN'):
            if phi == 0:
                phi = phi_test

        print('******************************', file=log)
        print('******************************', file=log)
        print('******************************', file=log)

    # CONFUSION MATRIX FOR NORMAL_2
    cf, losses, ground_truth, y_hat_n2 = tester({'NORMAL 2':data_ds3_t2_normal[:LIMITADOR]}, net, loss_function, phi)

    print('NORMAL 2', file=log)
    print(cf, file=log)
    print('******************************', file=log)
    print('******************************', file=log)
    print('******************************', file=log)

    for n, fault_name in enumerate(faults):

        datas = {'X_TEST': X_test[:LIMITADOR],
                 'fault_name': faults[fault_name][:LIMITADOR]}

        cf, losses, ground_truth, y_hat = tester(datas, net, loss_function, phi)

        print('X_TEST x ' + fault_name, file=log)
        print(cf, file=log)
        print('******************************', file=log)
        print('******************************', file=log)
        print('******************************', file=log)

    return y_hat_n2, phi

class Incremental_Stats():

    def __init__(self):
        self.count = 0
        self.mu = 0.0
        self.var = 0.0

    def add(self, x):
        if self.count == 0:
            new_mu = x
            new_var = 0
        else:
            new_mu = (self.count * self.mu + x) / (self.count + 1)
            new_var = ((self.count + 1) / self.count) * (((self.count * self.var) / (self.count + 1)) + (((x - new_mu)**2) / self.count))

        self.count = self.count+1
        self.mu = new_mu
        self.var = new_var

        return self.mu, self.var, self.count

    def get_mean(self):
        return self.mean

    def get_variance(self):
        return self.variance

    def get_std(self):
        return self.variance**0.5

    def get_n(self):
        return self.Elements

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data_ds3_t1_normal[:, :-1], data_ds3_t1_normal[:, -1], test_size=1-TRAIN_SIZE, random_state=42)

In [ ]:
architectures = [[54, 32, 9]]
last_layers = [nn.Tanh()]
batch_sizes = [32]
optims = ['RMSprop']
losses_fnc = [nn.L1Loss()]

'''last_layers = [nn.Sigmoid(), nn.Tanh()]
batch_sizes = [32]
optims = ['ADAM', 'SGD', 'RMSprop']
losses_fnc = [nn.BCEWithLogitsLoss(), nn.MSELoss() ,nn.L1Loss(), nn.BCELoss(), nn.SmoothL1Loss(), nn.PoissonNLLLoss()]'''

EPOCHS = 1
block_sizes = [1000]

for block_size in block_sizes:
    for optim in optims:
        for batch_size in batch_sizes:
            for last_layer in last_layers:
                for architecture in architectures:
                    for loss_fnc in losses_fnc:
                        print('next')

                        log = None
                        if FLUSH_FILE:
                            log = open(PATH_OUTPUTS+OUTPUT_FILE_NAME, "a", buffering=1)

                        #BCE só SIGMOID
                        if (type(loss_fnc) == type(nn.BCELoss()) and
                            type(last_layer) != type(nn.Sigmoid())):
                          continue

                        print('*****************************************', file=log)
                        print('****************************************', file=log)
                        print('************ HYPERPARAMETERS ************', file=log)
                        print('****LIMITADOR********************** ', LIMITADOR, file=log)
                        print('****LOSS FACTOR******************** ', LOSS_FACTOR, file=log)
                        print(architecture, file=log)
                        print(last_layer, file=log)
                        print(batch_size, file=log)
                        print(optim, file=log)
                        print(loss_fnc, file=log)
                        print('*****************************************', file=log)
                        print('*****************************************', file=log)
                        block_train = int(X_train.shape[0] / block_size)
                        net = []
                        count = 0
                        agg_loss = []
                        for n in np.arange(0, block_train):
                            print('.', end="")
                            count = count+1

                            if block_size*(n+1) < X_train.shape[0]:
                                data = X_train[block_size * n : block_size*(n+1)]
                            else:
                                data = X_train[block_size * n:]

                            net, output, loss, losses, loss_function = train(layers=architecture.copy(), last_layer=last_layer, lr=1e-3, epochs=EPOCHS, batch_size=batch_size, X_train=data, optim=optim, loss_fnc=loss_fnc, net=net)

                            losses_geral_step = generate_losses(data_test=X_train, net=net, loss_function=loss_fnc)
                            print('--')
                            print(np.mean(losses_geral_step))
                            print(np.std(losses_geral_step, ddof=1, dtype=np.float64))

                            losses_batch_step = generate_losses(data_test=data, net=net, loss_function=loss_fnc)
                            print('--')
                            print(np.mean(losses_batch_step))
                            print(np.std(losses_batch_step, ddof=1, dtype=np.float64))
                            agg_loss = np.concatenate((agg_loss, losses_batch_step), axis = 0)
                            phi_batch_step = np.mean(losses_batch_step) + np.std(losses_batch_step, ddof=1, dtype=np.float64)

                            print('--')
                            print(np.mean(agg_loss))
                            print(np.std(agg_loss, ddof=1, dtype=np.float64))

                            _, _ = test(X_train, X_test, net, loss_function, phi_batch_step)

                            print('***************************')
                            print('***************************')
                            print('***************************')
                            print('***************************')

                        phi = np.mean(losses_batch_step) + np.std(losses_batch_step, ddof=1, dtype=np.float64)
                        y_hat_n2, phi = test(X_train, X_test, net, loss_function, phi)

                        if FLUSH_FILE:
                            log.close()

next
.--
0.14637829109344683
0.46456087181483535
--
0.1378880844190717
0.44310426690322663
--
0.1378880844190717
0.44310426690322663


C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


***************************
***************************
***************************
***************************
.--
0.12468641990895978
0.4466808305448316
--
0.14598004795704037
0.5000322847971544
--
0.141934066188056
0.47232570066352436


C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


***************************
***************************
***************************
***************************
.--
0.11493981232347303
0.4394414952809974
--
0.14817176502011717
0.5623200676633926
--
0.1440132991320764
0.5040263484856445


...

***************************
***************************
***************************
***************************
.--
0.08047984280022016
0.4204497904108219
--
0.07341980226861779
0.4055714591870869
--
0.08515709874529039
0.42419960832067605


C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


***************************
***************************
***************************
***************************
.--
0.08123070275528961
0.42028752081512666
--
0.06683345421194099
0.3585137481588272
--
0.08500311853912779
0.42369207928867175


C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
C:\Users\MARCELO\anaconda3\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


***************************
***************************
***************************
***************************
.

In [ ]:
y_hat_n2, phi = test(X_train, X_test, net, loss_function)

np.nanstd

In [ ]:
cf, losses, ground_truth, y_hat = tester({'normal_1': data_ds3_t2_normal}, net, loss_function, loss_threshold = .6919097)

losses = generate_losses(X_train, net, loss_function)
np_losses = np.sort(np.array(losses))

In [ ]:
losses = np_losses[:int(len(np_losses)*1)]
print(np.mean(losses))
print(np.std(losses, ddof=1, dtype=np.float64))
print(np.median(losses))

In [ ]:
from scipy import stats, signal

In [ ]:
real = X_train[15]

In [ ]:
np.random.seed(42)
x = np.random.normal(size=1000)
plt.hist(losses, density=True, bins=200)  # `density=False` would make counts
plt.ylabel('Probability')
plt.xlabel('Data');

In [ ]:
shapiro_test = stats.shapiro(losses)
print(shapiro_test.statistic)
print(shapiro_test.pvalue)

In [ ]:




'''def incremental_stats(x):

    global K, Elements, Ex, Ex2
    if Elements == 0:
        K = x
    Elements += 1
    Ex += x - K
    Ex2 += (x - K) * (x - K)

    if (Elements>1):
        mean = K + Ex / Elements
        variance = (Ex2 - (Ex * Ex) / Elements) / (Elements - 1)

        return mean, variance

    return x, 0'''

inc_stats = Incremental_Stats()

print(inc_stats.add(2))
print(inc_stats.add(4))
print(inc_stats.add(6))
print(inc_stats.add(8))
print(inc_stats.add(10))

inc_stats = Incremental_Stats()
print(inc_stats.add(2))
print(inc_stats.add(4))

a = [2,4,6,8,10]
np.std(a, ddof=1)


In [ ]:
regenerate = net(torch.from_numpy(real.reshape(1,-1).astype(np.float32))).cpu().detach().numpy().reshape(-1,1)

In [ ]:
plt.plot(real, 'b')
plt.plot(regenerate, 'r')
plt.fill_between(np.arange(27), regenerate.reshape(1,-1)[0], real.reshape(1,-1)[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

In [ ]:
erros = (regenerate.reshape(1,-1)[0] - real.reshape(1,-1)[0])**2

In [ ]:
sum(erros)

## PHASE 2

In [ ]:
log = None
y_hat_n2, phi = test(X_train[:1000], X_test, net, loss_function)

In [ ]:
y_hat_n2 = np.array(X_train[:1000])

In [ ]:
eq0 = np.where(y_hat_n2 == 0)[0] #ok
eq1 = np.where(y_hat_n2 == 1)[0] #incorrect

print('ok', len(eq0)/len(X_train[:1000]), '%')
print('incorrect', len(eq1)/len(X_train[:1000]), '%')

In [ ]:
log = open(path+'/output_retrain.txt', "a", buffering=1)

X_train, X_test, y_train, y_test = train_test_split(data_ds3_t2_normal[eq1, :-1], data_ds3_t2_normal[eq1, :-1], test_size=0.2, random_state=42)

architectures = [[dimension,128,9]]
last_layers = [nn.Sigmoid()]
batch_sizes = [32]
optims = ['ADAM']
losses_fnc = [nn.BCEWithLogitsLoss()]

for architecture in architectures:
    for optim in optims:
        for batch_size in batch_sizes:
            for last_layer in last_layers:
                for loss_fnc in losses_fnc:

                    #BCE só SIGMOID
                    if (type(loss_fnc) == type(nn.BCELoss()) and
                            type(last_layer) != type(nn.Sigmoid())):
                        continue

                    print('*****************************************', file=log)
                    print('****************************************', file=log)
                    print('************ HYPERPARAMETERS ************', file=log)
                    print('*****************************************', file=log)
                    print('*****************************************', file=log)
                    print(architecture, file=log)
                    print(last_layer, file=log)
                    print(batch_size, file=log)
                    print(optim, file=log)
                    print(loss_fnc, file=log)
                    print('*****************************************', file=log)
                    print('*****************************************', file=log)

                    net, output, loss, losses, loss_function = train(layers=architecture.copy(), last_layer=last_layer, lr=1e-3, epochs=1, batch_size=batch_size, X_train=X_train, optim=optim, loss_fnc=loss_fnc, net=net)

                    _, phi = test(np.concatenate((X_train, data_ds3_t1_normal[:,:-1])), data_ds3_t1_normal, net, loss_function)

log.close()

In [ ]:
log = open(path+'/output_retrain.txt', "a", buffering=1)
_, phi = test(X_train, data_ds3_t1_normal, net, loss_function) #x_train, x_test, net, loss_fnc ### X_TRAIN precisa conter os dados históricos
log.close()

In [ ]:
cf, losses, ground_truth, y_hat = tester({'normal_1': data_ds3_t2_normal}, net, loss_function, loss_threshold = .6919097)

y_hat = np.array(y_hat)

eq0 = np.where(y_hat == 0)[0] #ok
eq1 = np.where(y_hat == 1)[0] #incorrect

print('normal', len(eq0)/len(y_hat), '%')
print('anomaly', len(eq1)/len(y_hat), '%')